<h1 style="text-align: center; font_size: 50px; color: green;">Song Recomender </h1>

In [4]:
from IPython.display import display, HTML

# Your HTML code with centered text
html_code = '<div style="text-align: center;">Your centered text goes here.</div>'

# Display the centered text in the Jupyter cell
display(HTML(html_code))


## Goal for today

look for differnt playlists/albums etc from varied genres
varied genres will improve the clustering machine learning we will do tomorrow

### Playlists
- everything good
- 80s
- hip hop


### Pseudo code

print welcome message

ask for input

if input is empty
    print("You didn't enter a song. Please try again."

### Ideas and Improvements
#### General
- Formatting text
    - Check for ways to nicely format text  with bolds, headings etc
    - check if buttons can be added
- Make function download latest hot 100 so it is always checking latest
- when not in hot 100 recommend song from alt 250 [done]
- ask would you like to enter another song? [done]
- user can choose number of recomendatiosn they would like
- Investigate how i could integrate work out data, heart rate, running speed etc
- Bonus feature: workout playlist creator: input genre, bpm, some sample songs and reccomender will recomend 10 songs for your workout playlist
- add plugins for jupyter lab to colapse code  boxes
- if user asks for a second recomendation ensure the same song is not recomendded [priority low]
  

#### Error handling
- If user enters an artists name, prompt them with "that is an artist not a song, we found this song by {artist} would you like to select this song?

- for fuzzy matching, ask user "did you mean..."
    -user selects y/n

## To do
#### Priority 
- embedd spotify player

#### Bonus
- research what each of the audio features do

#### Tidying up
- fix "would you like to continue" after providing hot 100 recommendation
- fix "did you mean" to allow user response (y/n), if no, prompt user for new input


user input

search for track id
get audio features
scale
predict cluster
recommend random song cluster


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
from fuzzywuzzy import process
from IPython.display import display, HTML
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame

In [4]:
hot_100_df = pd.read_csv('../data/hot_100.csv',index_col = 0)
alt_250 = pd.read_csv('../data/alt_250.csv',index_col = 0)


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [1]:
def song_recommender(hot_100_df, additional_songs_df):
    print("\n                                     Welcome to Gnod's Song Recommender!\nThe Recommender allows you to search for a song you like. It will then inform you if the song is in the Billboard Hot 100 or not and also provide you with a recomendation for another song\nNote: The song search is case insensitive.\n")
    while True:
        user_input = input("Please enter the name of a song.\n  Song: ").strip()
        if not user_input:
            print("\nYou didn't enter a song. Please try again.\n  Song: ")
            continue
        
        try:
            # Convert all song titles to lowercase for case-insensitive comparison
            songs_lower = hot_100_df['song'].str.lower()
            
            if user_input.lower() in songs_lower.values:
                print(f"\n'{user_input}' is in the Billboard Hot 100.")
                
                # Filter out the user's song and select a random recommendation
                hot_recommendations = hot_100_df[~songs_lower.str.contains(user_input.lower())]
                if not hot_recommendations.empty:
                    recommended_hot_song = random.choice(hot_recommendations['song'].values)
                    print(f"\nHere's another song you might like: '{recommended_hot_song}'")
                    cont = input("\nWould you like to select another song?\n  Response: ").lower()
                    if cont == "no":
                        print("\nThank you for using Gnod's Song Recommender\nGoodbye")
                        break
                break  # Exit loop after providing a recommendation
            
            else:
                
                # Use fuzzy matching to find the closest match to user_input in the 'song' column
                result = process.extractOne(user_input, songs_lower, score_cutoff=80)
                # score_cutoff is the minimum score threshold for considering a match
        
                if result: # A match was found with a score above the cutoff
                    closest_match, match_score, i = result
                    print(f"\nDid you mean '{closest_match}'? It is in the Billboard Hot 100.")
                    break
                else:
                    print(f"\n'{user_input}' is not in the Billboard Hot 100")
                    recommended_other_song = random.choice(additional_songs_df['song'].values)
                    print(f"\nHere's another song you might like: '{recommended_other_song}'")
                    
        except KeyError:
            print("\nError: The DataFrame does not have a 'song' column.")
            break
        
        cont = input("\nWould you like to select another song?\n  Response: ").lower()
        if cont == "no":
            print("\nThank you for using Gnod's Song Recommender\nGoodbye")
            break



In [6]:
results = sp.search(q="user_input",limit=1,market="GB")
track_id = results["tracks"]["items"][0]["id"]
track_id

'0KzAbK6nItSqNh8q70tb0K'

In [7]:
IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [2]:
song_recommender(hot_100_df,alt_250)


                                     Welcome to Gnod's Song Recommender!
The Recommender allows you to search for a song you like. It will then inform you if the song is in the Billboard Hot 100 or not and also provide you with a recomendation for another song
Note: The song search is case insensitive.



Please enter the name of a song.
  Song:  lovin on me



'lovin on me' is in the Billboard Hot 100.

Here's another song you might like: 'Monaco'



Would you like to select another song?
  Response:  test
